In [7]:
import geopandas as gpd
import pandas as pd
import pandas as pd
import geopandas
import gmaps
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon

%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time

import osmnx as ox
import networkx as nx

In [8]:
def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes


def splitDataFrameList(df,target_column,separator):
    import pandas
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pandas.DataFrame(new_rows)
    return new_df
    

In [9]:
merge_road = gp.GeoDataFrame.from_file('msia_route_weight/msia_route_weight.shp', geometry='geometry')
merge_road.head()

osm_id_x  code_x     fclass_x                    name_x   ref oneway  \
0  4931450    5122  residential  Jalan Pinggiran USJ 1/17  None      B   
1  4931468    5115     tertiary     Jalan Pinggiran USJ 1  None      B   
2  4931834    5122  residential  Persiaran Perusahaan U/1  None      B   
3  4931877    5113      primary            Jalan Montford    B9      F   
4  4934921    5113      primary          Bulatan Megawati  None      F   

   maxspeed  layer bridge tunnel  ... VARNAME__1 NL_NAME__2 TYPE_2_y  \
0         0      0      F      F  ...       None       None     None   
1         0      0      F      F  ...       None       None     None   
2         0      0      F      F  ...       None       None     None   
3         0      0      F      F  ...       None       None     None   
4         0      0      F      F  ...       None       None     None   

  ENGTYPE__1 CC_2_y HASC_2_y lat lon cx_count_i  \
0       None   None     None NaN NaN        0.0   
1       None   None     None NaN NaN        0.0   
2       None   None     None NaN NaN        0.0   
3       None   None     None NaN NaN        0.0   
4       None   None     None NaN NaN        0.0   

                                            geometry  
0  LINESTRING (101.5552027 3.0501994, 101.5562486...  
1  LINESTRING (101.5548825 3.0500569, 101.5550899...  
2  LINESTRING (101.5546656 3.091444, 101.5546525 ...  
3  LINESTRING (101.5533042 3.0878122, 101.5527241...  
4  LINESTRING (101.5435172 3.0521068, 101.5433673...  

[5 rows x 45 columns]

In [10]:
from shapely.geometry import shape
from shapely.ops import unary_union
import fiona
geoms =[shape(feature['geometry']) for feature in fiona.open("msia_routes_kl/msia_routes_kl.shp")]
weights = merge_road['cx_count_i']
import itertools
# create a Graph
import networkx as nx
G = nx.Graph()
counter = 0
for line in geoms:
    if line.geom_type== "MultiLineString":
        for line_part in line:
            #print(line_part)
            for seg_start, seg_end in zip(list(line_part.coords),list(line_part.coords)[1:]):
                G.add_nodes_from([seg_start, seg_end])
                G.add_edge(seg_start, seg_end, weight=float(weights[counter]))
                
    else:
        #print(line)
        #print(line.coords)
    
        for seg_start, seg_end in zip(list(line.coords),list(line.coords)[1:]):
            G.add_nodes_from([seg_start, seg_end])
            G.add_edge(seg_start, seg_end, weight=float(weights[counter])) 
            
    counter += 1

In [15]:
list(G.nodes)

[(101.5552027, 3.0501994),
 (101.5562486, 3.0500211),
 (101.556847, 3.0499153),
 (101.5569946, 3.0499087),
 (101.5572457, 3.0498558),
 (101.557493, 3.0497979),
 (101.5582923, 3.0496763),
 (101.5587598, 3.049596),
 (101.5588407, 3.0496144),
 (101.5590161, 3.0498156),
 (101.5592535, 3.0500724),
 (101.5597251, 3.050633),
 (101.5602334, 3.0512056),
 (101.5603128, 3.0514655),
 (101.5603465, 3.05174),
 (101.5604341, 3.0522776),
 (101.5548825, 3.0500569),
 (101.5550899, 3.0500209),
 (101.5551143, 3.0500166),
 (101.5552623, 3.0499909),
 (101.5568197, 3.0497217),
 (101.556873, 3.0497112),
 (101.5574543, 3.0495928),
 (101.5587815, 3.0493665),
 (101.5589368, 3.0493978),
 (101.5591139, 3.0495558),
 (101.5593955, 3.0498799),
 (101.5599363, 3.0504603),
 (101.5603733, 3.0509799),
 (101.5546656, 3.091444),
 (101.5546525, 3.0906138),
 (101.5546558, 3.0898678),
 (101.554657, 3.0895888),
 (101.5546444, 3.0889426),
 (101.5546217, 3.0871187),
 (101.5546244, 3.0862943),
 (101.5546173, 3.085729),
 (101.55463

In [ ]:
101.5028383, 3.0478604
101.8319226, 2.9729332

101.6072208, 3.0588637
101.6838389, 2.9979351


In [25]:
origin = (101.6072208, 3.0588637)
destination = (101.6838389, 2.9979351)

route1 = nx.shortest_path(G, origin, destination)
route2 = nx.dijkstra_path(G, origin, destination)
#print(route1)
#print(route2)
route1_arr_lat = []
route1_arr_lon = []
route2_arr_lat = []
route2_arr_lon = []

for element in route1:
    route1_arr_lat.append(element[1])
    route1_arr_lon.append(element[0])
    
for element in route2:
    route2_arr_lat.append(element[1])
    route2_arr_lon.append(element[0])


import geopandas as gpd
from shapely.geometry import Polygon
#print(len(route1_arr_lat))
#print(len(route1_arr_lon))
#print(route1_arr_lat)
#print(route1_arr_lon)
polygon_geom = LineString(zip(route1_arr_lon, route1_arr_lat))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
polygon['geohash_set'] = np.nan
polygon['geohash_set'] = polygon.apply(lambda x: compute_geohash_tiles_from_polygon(x.geometry, 7), axis=1)
polygon['geohash_set'] = polygon['geohash_set'].astype(str)
polygon['geohash_set'] = polygon['geohash_set'].str[1:]
polygon['geohash_set'] = polygon['geohash_set'].str[:-1]
polygon = splitDataFrameList(polygon,'geohash_set',',')
polygon = geopandas.GeoDataFrame(polygon, geometry='geometry')
polygon.to_file(filename='msia_route_1/route1.shp')

polygon_geom = LineString(zip(route2_arr_lon, route2_arr_lat))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
polygon['geohash_set'] = np.nan
polygon['geohash_set'] = polygon.apply(lambda x: compute_geohash_tiles_from_polygon(x.geometry, 7), axis=1)
polygon['geohash_set'] = polygon['geohash_set'].astype(str)
polygon['geohash_set'] = polygon['geohash_set'].str[1:]
polygon['geohash_set'] = polygon['geohash_set'].str[:-1]
polygon = splitDataFrameList(polygon,'geohash_set',',')
polygon = geopandas.GeoDataFrame(polygon, geometry='geometry')
polygon.to_file(filename='msia_route_2/route2.shp')


In [26]:
route2

[(101.6072208, 3.0588637),
 (101.6071752, 3.0588262),
 (101.6070679, 3.0588208),
 (101.606805, 3.0588864),
 (101.6058984, 3.0590404),
 (101.6064703, 3.0588554),
 (101.6066788, 3.0587939),
 (101.6072234, 3.0586333),
 (101.6074514, 3.0585798),
 (101.6077223, 3.0585101),
 (101.6081649, 3.0583923),
 (101.6082963, 3.0583333),
 (101.6084272, 3.058273),
 (101.6094121, 3.0578191),
 (101.6099293, 3.0575514),
 (101.6107075, 3.0569539),
 (101.6115831, 3.0562441),
 (101.6118992, 3.0559986),
 (101.612092, 3.0558711),
 (101.6123505, 3.0557209),
 (101.6127019, 3.0555421),
 (101.6129285, 3.0554402),
 (101.6131789, 3.0553497),
 (101.6134309, 3.0552745),
 (101.613894, 3.0551862),
 (101.6149347, 3.0551326),
 (101.6154362, 3.0552301),
 (101.6159325, 3.0553553),
 (101.6165665, 3.0556041),
 (101.61703, 3.0558487),
 (101.6174009, 3.0561192),
 (101.6177537, 3.0564022),
 (101.6182353, 3.0569303),
 (101.6192141, 3.0581627),
 (101.6201446, 3.0592287),
 (101.6196238, 3.0584675),
 (101.6194849, 3.0581673),
 (101.6

In [27]:
#route1
with open('msia_route1_2nd.txt', 'w') as f:
    for item in route1:
        f.write("%s\n" %str(item))

In [28]:
#route2
with open('msia_route2_2nd.txt', 'w') as f:
    for item in route2:
        f.write("%s\n" %str(item))